<a href="https://colab.research.google.com/github/PoChihKuo/2019_aarhus_critical_data/blob/master/Decision_tree_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HW2: Decision Tree** 


Predict the patients' death (hospDIED) in the *MIMIC* dataset using decision tree

## Load the Example Data
First, load the *Restaurant waiting* dataset: **data.csv**

In [ ]:
example = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/data.csv')
example

,Unnamed: 0,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,X1,T,F,F,T,Some,High,F,T,French,8,T
1,X2,T,F,F,T,Full,Low,F,F,Thai,40,F
2,X3,F,T,F,F,Some,Low,F,F,Burger,8,T
3,X4,T,F,T,T,Full,Low,F,F,Thai,12,T
4,X5,T,F,T,F,Full,High,F,T,French,70,F
5,X6,F,T,F,T,Some,Medium,T,T,Italian,3,T
6,X7,F,T,F,F,None,Low,T,F,Burger,7,F
7,X8,F,F,F,T,Some,Medium,T,T,Thai,6,T
8,X9,F,T,T,F,Full,Low,T,F,Burger,80,F
9,X10,T,T,T,T,Full,High,F,T,Italian,20,F


In [ ]:
example = example.drop(['Unnamed: 0'], axis=1)

In [ ]:
# change the string categorical to integer labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

example['Alternate'] = labelencoder.fit_transform(example['Alternate'])
example['Bar'] = labelencoder.fit_transform(example['Bar'])
example['Friday'] = labelencoder.fit_transform(example['Friday'])
example['Hungry'] = labelencoder.fit_transform(example['Hungry'])
example['Patrons'] = labelencoder.fit_transform(example['Patrons'])
example['Price'] = labelencoder.fit_transform(example['Price'])
example['Raining'] = labelencoder.fit_transform(example['Raining'])
example['Reservation'] = labelencoder.fit_transform(example['Reservation'])
example['Type'] = labelencoder.fit_transform(example['Type'])

example

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
2,0,1,0,0,2,1,0,0,0,8,T
3,1,0,1,1,0,1,0,0,3,12,T
4,1,0,1,0,0,0,0,1,1,70,F
5,0,1,0,1,2,2,1,1,2,3,T
6,0,1,0,0,1,1,1,0,0,7,F
7,0,0,0,1,2,2,1,1,3,6,T
8,0,1,1,0,0,1,1,0,0,80,F
9,1,1,1,1,0,0,0,1,2,20,F


## Calculating the Entropy *(20%)*


In [ ]:
# split the data by given attribute and its threshold
def partition(data, column, threshold):
  """
  The *partition* function will split the input data into 2 branches.
    args:
    *   data(DataFrame): the input data
    *   column(str): the attribute(column name)
    *   threshold(float): the column's threshold for splitting the data
    returns:
    *   match_branch(DataFrame): the divided data that matches the assigned column's threshold
    *   false_branch(DataFrame): the divided data that doesn't match the assigned column's threshold
  """
  match_branch = data[data[column]<=threshold]
  false_branch = data[data[column]>threshold]
  return match_branch, false_branch

If we try *partition(example, 'Patrons', 1.5)*:

split example to 

match_left(Patrons<= 1.5) (Patrons != Some) and  
false_right(Patrons > 1.5) (Patrons == Some)

column: Patrons, threshold: 1.5



In [ ]:
match_left, false_right = partition(example, 'Patrons', 1.5)
print(match_left.shape)
print(false_right.shape)
match_left.head()

(8, 11)
(4, 11)


,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
1,1,0,0,1,0,1,0,0,3,40,F
3,1,0,1,1,0,1,0,0,3,12,T
4,1,0,1,0,0,0,0,1,1,70,F
6,0,1,0,0,1,1,1,0,0,7,F
8,0,1,1,0,0,1,1,0,0,80,F


In [ ]:
def entropy(data):
  """
  The *entropy* function will calculate the entropy of the node(data)
  args:
  *    data(DataFrame): the data you're calculating for the entropy
  output:
  *    entropy(float): the node(data)'s entropy     
  """
  
  waiting_p = data[data['Wait']=='T'].size / data.size
  waiting_q = data[data['Wait']=='F'].size / data.size
  if(waiting_p == 0 or waiting_q == 0): 
    entropy = - 1 * np.log2(1)
  else:
    entropy = - waiting_p * np.log2(waiting_p) - waiting_q * np.log2(waiting_q)
  return entropy

An example of the output from entropy (Use this to check if your output is correct): 

In [ ]:
match, false = partition(example, 'Hungry', 0.5)
entropy1 = entropy(match)
print(entropy1)

0.7219280948873623


In [ ]:
#implementation 1: calculate the entropy of 'false_right' data
ans_entropy = entropy(false)

## Find the Best Split *(20%)*

In [ ]:
# search for the best attribute and the value(threshold) to split the data
def findBestSplit(data):
    """
    The *findBestSplit* function finds the best combination of attribute and value(with the largest reduction in entropy) to split the data.
    args:
    *   data(DataFrame): the data you try to split(build the decision tree)
    output:
    *   column_best(str): the attribute(column) split with the largest reduction in entropy
    *   value_best(float): the value(threshold) of the column_best attribute to split the data
    """
    all_entropy = 9999        #the overall entropy : save the value of the smallest entropy
    column_best = 0
    value_best = 0

    if(entropy(data) == 0):
      return column_best, value_best

    # Calculate all combinations

    # column_temp = np.array(['Alternate', 'Bar', 'Friday', 'Hungry', 'Patrons', 'Patrons', 'Price', 'Price', 'Raining',	'Reservation', 'Type', 'Type', 'Type', 'WaitEstimate', 'WaitEstimate', 'WaitEstimate', 'WaitEstimate', 'WaitEstimate', 'WaitEstimate', 'WaitEstimate', 'WaitEstimate'])
    # value_temp = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 1.5, 0.5, 1.5, 0.5, 0.5, 0.5, 1.5, 2.5, 4.5, 6.5, 7.5, 10, 16, 30, 55, 75])
    # total_number = column_temp.size

    value_temp = []
    column_temp = []
    column_name = np.array(['Alternate', 'Bar', 'Friday', 'Hungry', 'Patrons', 'Price', 'Raining', 'Reservation', 'Type', 'WaitEstimate'])

    for j in range(0, column_name.size):
      data_sort = data
      data_sort = data_sort.sort_values(by=[column_name[j]])
      data_sort = np.array(data_sort[column_name[j]].to_numpy())
      for i in range(1, data_sort.size):
        if(data_sort[i] != data_sort[i-1]):
          value_temp.append((data_sort[i]+data_sort[i-1])/2)
          column_temp.append(column_name[j])
    
    total_number = np.size(column_temp)
    # print(value_temp, column_temp)

    # Find the best combination
    for i in range(0, total_number):
      match_temp, false_temp = partition(data, column_temp[i], value_temp[i])
      if(match_temp.empty):
        entropy_temp_2 = entropy(false_temp)
        entropy_temp = entropy_temp_2*false_temp['Wait'].size / data['Wait'].size
      elif(false_temp.empty):
        entropy_temp_1 = entropy(match_temp)
        entropy_temp = entropy_temp_1*match_temp['Wait'].size / data['Wait'].size
      else:
        entropy_temp_1 = entropy(match_temp)
        entropy_temp_2 = entropy(false_temp)
        entropy_temp = (entropy_temp_1*match_temp['Wait'].size + entropy_temp_2*false_temp['Wait'].size) / data['Wait'].size
      
      if(entropy_temp < all_entropy):
        all_entropy = entropy_temp
        column_best = column_temp[i]
        value_best = value_temp[i]
      # print(entropy_temp)
      # print(column_temp[i], value_temp[i])

    
    # print(all_entropy)
    return column_best, value_best
  

An example of the output from findBestSplit (Use this to check if your output is correct):

In [ ]:
column_best, value_best = findBestSplit(match)
print(column_best)
print(value_best)

Patrons
1.5


In [ ]:
#implementation 2: Find the best split of the 'match' data
ans_column, ans_value = findBestSplit(false)

## Decision Tree Building *(30%)*

Use the above functions to help building the decision tree





In [ ]:
def buildTree(df, depth):
  """
  The *buildTree* function builds the decision tree
  args:
  *     df(DataFrame): the data you want to apply the decision tree
  *     depth(int) : the depth of your tree
  output:
  *     decisionSubTree(dict): the decision tree structure including root, branch, leaf(with the attributes and thresholds)
  *     features(list): the features(attributes) name in the decision tree structure(from root to branch and leaf)
  *     thresholds(list): the corresponding thresholds for the features in the 'features' list
  """
  features = []
  thresholds = []
  decisionSubTree = {}
  if(depth != 0):
    data = df
    column_best, value_best = findBestSplit(data)
    if(column_best==0):
      decisionSubTree = ('T')
    else:
      features.append(column_best)
      thresholds.append(value_best)

      match, false = partition(data, column_best, value_best)
      decisionSubTree_m, features_m, thresholds_m = buildTree(match, depth-1)
      decisionSubTree_f, features_f, thresholds_f = buildTree(false, depth-1)
      if(features_m != []):
        features.extend(features_m)
        thresholds.extend(thresholds_m)
      if(features_f != []):
        features.extend(features_f)
        thresholds.extend(thresholds_f)
      
      key_1 = str(column_best) + ' <= ' + str(value_best)
      if(decisionSubTree_m == 'T'):
        key_2_m = decisionSubTree_m
      else:
        key_2_m = ('F', features, thresholds)

      if(decisionSubTree_f == 'T'):
        key_2_f = decisionSubTree_f
      else:
        key_2_f = ('F', features, thresholds)

      decisionSubTree = {key_1: [key_2_m, key_2_f]}
    
    
  return decisionSubTree, features, thresholds
  

An example of the output from buildTree (Use this to check if your output is correct): 

In [ ]:
tree, features, thresholds= buildTree(example, 2)
print(tree)
print(features)
print(thresholds)

{'Patrons <= 1.5': [('F', ['Patrons', 'Hungry'], [1.5, 0.5]), 'T']}
['Patrons', 'Hungry']
[1.5, 0.5]


In [ ]:
#implementation 3: decision tree building (depth=10)
ans_tree, ans_features, ans_thresholds = buildTree(example,10)

In [ ]:
#save implementation csv
ans_path = STUDENT_ID + '_implementation.csv'

imp = []
imp.append(ans_entropy)
imp.append(ans_column)
imp.append(ans_value)
for i in range(len(ans_features)):
  imp.append(ans_features[i])
for m in range(len(ans_thresholds)):
  imp.append(ans_thresholds[m])
print(imp)
pd.DataFrame(imp).to_csv(ans_path, header = None, index = None)

[0.863120568566631, 'WaitEstimate', 16.0, 'Patrons', 'Hungry', 'Friday', 'Price', 1.5, 0.5, 0.5, 0.5]


# **2. Classification with the MIMIC Dataset**
In the second part, you need to classify **'hospDIED'(death)** in the MIMIC dataset.

Please put the classification result in a csv file. (**[STUDENTID]_prediction.csv**)

**Note:** Decision tree is recommended but not mandatory.

## Prediction (Performance) *(20%)*

The **y_test** (hospDIED) of this data is hidden, you need to use the x_test.csv to predict the y_test.

**Note:** You **can** now import the packages you need here!

Get the MIMIC data here!

Data Description: 

[Data Description](https://docs.google.com/spreadsheets/d/1pxqxQFhIcv_hrgWEtwhXE6zBVQ5ISa-13PIhvXMtWCY/edit?usp=sharing) (You can find the data description here.)

**Note:**
*   You can select the features you want to use.
*   You can use any ML models to predict the y_test.

  (However, there is a 10% bonus if you **visualize** the decision tree in this part)


[ref]: *Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.*


In [ ]:
#Read data
x_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/y_train.csv')
x_train.head()
x_train = x_train.drop(['indextime', 'ethnicity'], axis=1)

Complete your model with validation:

In [ ]:
#Split the data into training and validation sets
#Build you model and evaluate it on your validation set

from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score 

x_training, x_validation, y_training, y_validation = train_test_split(x_train, y_train, test_size = 0.3, stratify=y_train)

decision_tree_clf = DecisionTreeClassifier(criterion='entropy')
decision_tree = decision_tree_clf.fit(x_train, y_train)
y_prediction = decision_tree.predict(x_validation)

print(y_prediction)
print(y_validation['hospDIED'].to_numpy())
accuracy = accuracy_score(y_prediction, y_validation['hospDIED'].to_numpy())
print(accuracy)

[0 0 1 ... 0 0 0]
[0 0 1 ... 0 0 0]
1.0


Make the final prediction from your model:

In [ ]:
x_test = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_test.csv')
x_test = x_test.drop(['indextime', 'ethnicity'], axis=1)
# y_pred = 

In [ ]:
#final prediction with your decision tree(optional)
def treePredictions(df, d_tree):
  
  predictions = d_tree.predict(df)
  return predictions

y_pred = treePredictions(x_test, decision_tree)

To export your predcition as a CSV file and hand in the CSV on elearn

In [ ]:
output_path = STUDENT_ID + '_prediction.csv'

tree_test_pred = pd.DataFrame( {'subject_id': x_test.subject_id,
                  'prediction': y_pred } )
tree_test_pred.to_csv(output_path, index = False)

## Visualizing the Decision Tree *(10% bonus)*

**Note:** Save the visualization result image as **[STUDENT_ID]_visualization.png**

*   Your visualization image of the decision tree can contain **five** layers at most.


In [ ]:
#Decision Tree Visualization
from graphviz import Source
# dot_data = export_graphviz(decision_tree, out_file=None) 
# graph = graphviz.Source(dot_data)

dead_or_not = ['0', '1']
hosp_feature = list(x_train.columns)

graph = Source(tree.export_graphviz(decision_tree, out_file=None, max_depth=4, filled=True, rounded=True, special_characters=True, feature_names=hosp_feature, class_names=dead_or_not))
graph.format = 'png'
graph.render("107062317_visualization", view=True) 

'107062317_visualization.png'

# Report *(10%)*

Report should be submitted as a pdf file! (**[STUDENT_ID]_report.pdf**)

*   List the top 3 splitting features and their thresholds of your model
*   Briefly describe how you build the decision tree
*   Describe if you apply any improvement on your decision tree model
*   If you preprocess the MIMIC data in the second part (selecting features...), describe the work and reasons
*   Summarize your work
*   Do not exceed 2 pages!






# Save the Code File
Please save your code and submit it as an ipynb file! (**[STUDENT_ID]_hw2.ipynb**)